In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

# SETUP

In [2]:
# # Number of nodes
# num_nodes = 1000
# sync_convergence_threshold = 1*10**-8

# # starting mae for convergence
# starting_mae_for_simulation = 10

# # RUN FIRST TIME
# sum_of_iterations = 0
# num_iterations = 1
# for _ in range(1000):
#     A,b = data_gen.get_1D_line_matrix_PSD_difficult(num_nodes, eigenvalue_spread=1e-5, regularization_strength=1e-2, noise_strength=1e-2)
#     graph = NetworkxGraph(A)
#     P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     final_mean = list(mu_i)
#     final_std = P_i
#     if iteration < 1000:
#         break
#     else:
#         print("=========== RESTART ===========")

# print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [2]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    A,b = data_gen.get_2D_lattice_matrix_PSD_difficult(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)), eigenvalue_spread=0.1, regularization_strength=1e-2, noise_strength=1e-2)
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 1000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
1.11721363596054
-----
iteration: 2
0.2199055166451334
-----
iteration: 3
0.09701180291514114
-----
iteration: 4
0.04450800053925566
-----
iteration: 5
0.018094423598210688
-----
iteration: 6
0.00743402713314304
-----
iteration: 7
0.002822052175282901
-----
iteration: 8
0.0010602912918912088
-----
iteration: 9
0.00041028994585261605
-----
iteration: 10
0.00014918586407123183
-----
iteration: 11
5.3839151233577416e-05
-----
iteration: 12
1.9860810772797938e-05
-----
iteration: 13
7.5605590315052384e-06
-----
iteration: 14
2.8837115838875683e-06
-----
iteration: 15
1.1416662519799444e-06
-----
iteration: 16
4.3901562281418546e-07
-----
iteration: 17
1.7504900879369274e-07
-----
iteration: 18
6.664001874625056e-08
-----
iteration: 19
2.5809568279517847e-08
-----
iteration: 20
0.0
-----
AVE. SYNC ITERATIONS = 20.0


### 

In [3]:
async_convergence_threshold = 1*10**-5
sum_of_iterations = 0
num_iterations = 1
NODE_UPDT_PE = 1
PEs = 1
for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, STARTING_MAE = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")
print(STARTING_MAE)

iteration: 1
0.38812298662709255
-----
iteration: 2
0.3881243736459922
-----
------------ EARLY EXIT ------------
-------------- ITERATION = 1 => Streams = 3 --------------
AVE. ASYNC ITERATIONS = 3.0
0.3881229866270925


In [4]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    b = starting_mae_for_simulation * b / STARTING_MAE
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=200, mae=False, convergence_threshold=sync_convergence_threshold, show=False)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

AVE. SYNC ITERATIONS = 21.0


### START SIMULATION

In [5]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 97
PEs = 3

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 1000
    while iteration > 1000-1:
        P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
10.000000000000002
-----
iteration: 2
7.880660775237701
-----
iteration: 3
6.0283546340272025
-----
iteration: 4
4.808456540203095
-----
iteration: 5
3.8027193382058098
-----
iteration: 6
2.9167488458668895
-----
iteration: 7
2.4816102365663983
-----
iteration: 8
1.9151914507657093
-----
iteration: 9
1.6348737855630096
-----
iteration: 10
1.1447309160716186
-----
iteration: 11
0.884912798873321
-----
iteration: 12
0.6528726697324228
-----
iteration: 13
0.5192075690877419
-----
iteration: 14
0.4196790055498835
-----
iteration: 15
0.3328156931716891
-----
iteration: 16
0.1890952925142021
-----
iteration: 17
0.14617932198642788
-----
iteration: 18
0.126774737932562
-----
iteration: 19
0.09067899352524926
-----
iteration: 20
0.07150579531621902
-----
iteration: 21
0.05960177710881861
-----
iteration: 22
0.04805818460942065
-----
iteration: 23
0.04465327941449064
-----
iteration: 24
0.03866102570972027
-----
iteration: 25
0.029827492030358872
-----
iteration: 26
0.0278915997437

In [7]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 1


# NODE_UPDT_PE = 97
# PEs = 2

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     iteration = 250
#     while iteration > 250-1:
#         P_i, mu_i, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(A, b, ratio=1.0, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#         break
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [6]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 97
PEs = 2

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 1000
    while iteration > 1000-1:
        P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
2.4992701457052586
-----
iteration: 2
2.2432896718180366
-----
iteration: 3
1.9167720864676128
-----
iteration: 4
1.643938493394265
-----
iteration: 5
1.3229556331924477
-----
iteration: 6
1.0623274826844762
-----
iteration: 7
0.8662866871009849
-----
iteration: 8
0.6594237111170087
-----
iteration: 9
0.5090195787065108
-----
iteration: 10
0.35137379808297
-----
iteration: 11
0.2531910102225533
-----
iteration: 12
0.18686317198645508
-----
iteration: 13
0.13517580259863154
-----
iteration: 14
0.10569376637638402
-----
iteration: 15
0.07657968242252523
-----
iteration: 16
0.06132183759680532
-----
iteration: 17
0.04503853192218965
-----
iteration: 18
0.034161657580900714
-----
iteration: 19
0.024660403156842096
-----
iteration: 20
0.019169649730171174
-----
iteration: 21
0.01402288992424634
-----
iteration: 22
0.010834590508924723
-----
iteration: 23
0.008115352983591168
-----
iteration: 24
0.0063497194018847635
-----
iteration: 25
0.004226465164309751
-----
iteration: 26
0

In [9]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 1


# NODE_UPDT_PE = 97
# PEs = 2

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     iteration = 250
#     while iteration > 250-1:
#         P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#         break
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [13]:
# async_convergence_threshold = 1*10**-5

# sum_of_iterations = 0
# num_iterations = 1


# NODE_UPDT_PE = 97
# PEs = 2

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     iteration = 1000
#     while iteration > 1000-1:
#         P_i, mu_i, iteration, T_max_lst, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
#         break
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [14]:
# PADDING_ANALYTICS